### Inicialização do ambiante

Para testar o script de geração do mapa de calor, vamos gerar pontos de calor randomicos utlizando as coordenadas da cidade de Blumenau.

Latitude e longitude vem de http://bboxfinder.com/#-27.007138,-49.159698,-26.674459,-48.965378

Acurária do 3º decimal é de ~100m
https://gis.stackexchange.com/questions/8650/measuring-accuracy-of-latitude-and-longitude

In [1]:
# Carrega bibliotecas utilizadas
import folium
import pandas as pd
from folium import plugins
import geopy
import string

In [2]:
ARQUIVO_DADOS = "/home/jvanz/vulneraveis/dados.csv"

ARQUIVO_COORDENADAS = "/home/jvanz/vulneraveis/dados_coordenadas.csv"

# limpeza dos dados
dados = pd.read_csv(ARQUIVO_DADOS, sep=";")
shape_anterior = dados.shape

# ajusta ceps. Remove os "-" e mantem apenas aquelas com 8 digitos
dados['cep'] = dados["cep"].astype(str).map(lambda x: x.strip().replace('-','')).map(lambda x: x if len(x) == 8 else "")

# ajusta logradouro. Remove espaços em brancos e pontuações na extremidades do texto
dados['logradouro'] = dados["logradouro"].astype(str).map(lambda x: x.strip(string.whitespace + string.punctuation))
dados = dados.loc[dados["logradouro"] != "nan"]

#ajusta nome do bairro
dados["bairro"] = dados["bairro"].astype(str).map(lambda x: x.strip(string.whitespace + string.punctuation))
dados.loc[dados["bairro"] == "nan", "bairro"] = ""

# ajusta numero. Permite apenas numeros
def ajusta_numero(numero):
    """Permite apenas numeros com digitos"""
    numero = numero.strip(string.whitespace + string.punctuation)
    return numero if numero.isnumeric() else ""
    
dados['numero'] = dados["numero"].astype(str).map(ajusta_numero)

print(f"{shape_anterior[0] - dados.shape[0]} registros removidos")


555 registros removidos


In [4]:


#deseja carregar as coordenadas?
carrega_coordenada = True
GOOGLE_API_KEY = "COLOQUE SUA CHAVE DE API AQUI"

localizador = geopy.geocoders.GoogleV3(GOOGLE_API_KEY)

def get_coordenadas(linha):
    """Função que busca as coordenadas utilizando o localizador definido."""
    if linha["latitude"] and linha["latitude"] != 0 and linha["longitude"] and linha["longitude"] != 0:
        # não precisamos buscar coordenadas que já temos
        return linha
    localizacao = localizador.geocode(linha["endereco_completo"])
    if localizacao:
        linha["latitude"] = localizacao.latitude
        linha["longitude"] = localizacao.longitude
    return linha

def monta_endereco_completo(linha):
    """Monta linha do endereço complete conforme dados disponiveis."""
    endereco = dados["logradouro"]
    endereco += "," + linha["numero"] if len(linha["numero"]) > 0 else ""
    endereco += "," + linha["bairro"] if len(linha["bairro"]) > 0 else ""
    endereco += ", Blumenau, Santa Catarina, Brasil"
    linha["endereco_completo"] = endereco
    return linha



# Precisa buscar as coordenadas novamente?
if carrega_coordenada:
    # colocar valores default para as coordenadas
    dados["latitude"] = 0
    dados["longitude"] = 0
    # monta um endereco completo para ser utlizado pelo buscador de coordenadas
    dados = dados.apply(monta_endereco_completo, axis=1)
    # procura latitude e longitude dos endereços
    dados = dados.apply(get_coordenadas, axis=1)
    # grava o arquivo com as coordenadas
    dados.to_csv(ARQUIVO_COORDENADAS,sep=";", index=False)

### Geração do mapa de calor

In [ ]:
# Carrega coordenadas dos pontos de calor do arquivo
coordenadas = pd.read_csv(ARQUIVO_COORDENADAS, sep=";")
coordenadas = coordenadas[["latitude", "longitude"]]
# ignore coordenadas invalidas
coordenadas = coordenadas[(coordenadas.latitude != 0) & (coordenadas.longitude != 0)]

long_min =  coordenadas.longitude.min()
long_max =  coordenadas.longitude.max()
lat_min = coordenadas.latitude.min()
lat_max = coordenadas.latitude.max()

# Plota o gráfico do mapa de calor
m = folium.Map(
    [np.mean([lat_min, lat_max]), np.mean([long_min, long_max])],
    zoom_start=11
)

m.add_child(
    plugins.HeatMap(coordenadas, radius=15)
)

m